In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

import time
import re
import pandas as pd

from tqdm.notebook import tqdm
from collections import defaultdict

from bs4 import BeautifulSoup
import requests

## Что нужно собрать:
* Гиперссылка на проект
* Название проекта
* Организация
* Конкурс
* Запрашиваемая сумма
* Cофинансирование
* Общая сумма расходов на реализацию проекта
* Цель
* Краткое описание
* Статус проекта
* Номер заявки (индекс таблицы)
* Дата подачи
* Сроки реализации
* ИНН
* ОГРН

In [2]:
start_url = 'https://президентскиегранты.рф/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=36568399-3aff-45f4-b418-d5362caf4595&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false'

In [30]:
browser = webdriver.Chrome(ChromeDriverManager().install(), ) # options=chrome_options
timeout = 10

In [5]:
def scrape_links():
    table_project = browser.find_element_by_xpath("//div[@class='table table--p-present table--projects']")
    project_links = []
    for i in table_project.find_elements_by_tag_name('a'):
        project_links.append(i.get_attribute('href'))
    return project_links

In [32]:
def scrape_elems(link):
    
    # load new page
    browser.get(link)
    WebDriverWait(browser, timeout).until(
                    lambda browser: browser.find_element(By.XPATH, "//div[@id='winner-aims']"))
    time.sleep(2)
    
    status = browser.find_element_by_xpath("//p[@class='winner-info__status']//span").text
    title = browser.find_element_by_tag_name('h2').text
    
    # сбор информации в content winner
    info = browser.find_element_by_xpath("//ul[@class='winner-info__list winner-info__item']")
    info_elems = info.find_elements_by_xpath("//span[@class='winner-info__list-item-text']")
    
    # если проект получил финансирование, то в контенте есть поле рейтинга, которое не нужно
    if len(info_elems) == 8:
        competition_year = info_elems[0].text
        grant_direction = info_elems[1].text
        app_number = info_elems[2].text
        app_date = info_elems[3].text
        implementation_terms = info_elems[4].text
        organization = info_elems[5].text
        inn = info_elems[6].text
        ogrn = info_elems[7].text
    
    else:
        competition_year = info_elems[0].text
        grant_direction = info_elems[1].text
        app_number = info_elems[3].text
        app_date = info_elems[4].text
        implementation_terms = info_elems[5].text
        organization = info_elems[6].text
        inn = info_elems[7].text
        ogrn = info_elems[8].text
    
    # сбор информации с кратким описанием проекта
    summary = browser.find_element_by_xpath("//div[@id='winner-summary']").text.split('\n')[1]
    target = browser.find_element_by_xpath("//div[@id='winner-aims']").find_element_by_tag_name('li').text
    
    return status, title, link, competition_year, grant_direction, app_number, app_date, \
            implementation_terms, organization, inn, ogrn, summary, target

In [33]:
def project_data_to_df(data, out_df):
    cols = ['status', 'title', 'links', 'competition_year', 'grant_direction', 'app_number', 'app_date', \
            'implementation_terms', 'organization', 'inn', 'ogrn', 'summary', 'target']
    if out_df is None:
        out_df = pd.DataFrame(data, columns=cols)
    else:
        out_df = pd.concat([out_df, pd.DataFrame(data, columns=cols)])
    return out_df

In [41]:
proj_data = []
# final_df = None

for page in tqdm(range(99, 191)): # 89 -> 191
    start_url = f'https://президентскиегранты.рф/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=36568399-3aff-45f4-b418-d5362caf4595&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false&page={page}'
    
    browser.get(start_url)
    WebDriverWait(browser, timeout).until(
                        lambda browser: browser.find_element(By.XPATH, "//div[@class='table table--p-present table--projects']"))

    time.sleep(2)
    links = scrape_links()

    for link in tqdm(links):
        data = scrape_elems(link)
        proj_data.append(data)
    
    final_df = project_data_to_df(proj_data, final_df)
    final_df.to_csv('data/project_grants_data.csv')
    proj_data.clear()

  0%|          | 0/181 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [44]:
final_df.shape[0] / 20

98.0

In [50]:
final_df.sample(5)

,status,title,links,competition_year,grant_direction,app_number,app_date,implementation_terms,organization,inn,ogrn,summary,target
15,проект не получил поддержку,TAJU - портал о нематериальном культурном насл...,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-1-010440,13.10.2021,01.02.2022 - 31.12.2022,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР К...",1001350120,1201000006459,Будущий 2022 год объявлен Президентом РФ В.В. ...,Содействие сохранению и популяризации народног...
1,проект не получил поддержку,Друзья,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-1-017331,15.10.2021,01.03.2022 - 31.05.2022,БЕЛГОРОДСКАЯ МЕСТНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ...,3123449271,1193100000048,Важным институтом социализации наряду с семьей...,Сохранение и укрепление межнационального мира ...
9,победитель конкурса,В дружбе народов - единство России,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2019,Укрепление межнационального и межрелигиозного ...,19-2-010658,31.07.2019,01.11.2019 - 30.09.2020,"Территориальное общественное самоуправление ""Т...",3416004579,1093400006864,Миграционные процессы как неотъемлемая часть ж...,Создание условий для формирования уважения к и...
9,проект не получил поддержку,Создание татарского этнокультурного кластера Р...,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2020,Укрепление межнационального и межрелигиозного ...,20-2-004536,23.03.2020,01.08.2020 - 31.12.2021,"ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ""САСОВСКАЯ МЕСТНАЯ ТА...",6234990160,1116200000982,"Проект представляет собой комплекс мер, направ...","Создание комплекса мер, направленных на сохран..."
3,проект не получил поддержку,Москва добрососедская,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2020,Укрепление межнационального и межрелигиозного ...,20-1-031211,26.11.2019,02.03.2020 - 30.11.2020,Региональная общественная организация - Татарс...,7705232290,1037700106201,Проект является продолжением реализованного в ...,Организация комплексной и системной работы по ...


In [20]:
df = pd.DataFrame(proj_data, columns=cols)

In [40]:
df.duplicated().sum()

0

In [27]:
df.to_csv('data/sample.csv')

In [43]:
(89 + 9) *20

1960

In [49]:
pd.read_csv('data/project_grants_data.csv', index_col=0).shape

(1960, 13)

In [28]:
for page in range(1, 191): print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
